In [1]:
from utils import plot_history, plot_result, calc_mean_image_metrics, predict_all
from losses import ssim_l2

In [8]:
import numpy as np

import os

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, Add, Subtract
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras import callbacks
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
images_clear_tt = np.load('images_clear_tt.npy')
images_noised_tt = np.load('images_noised_tt.npy')
images_clear_val = np.load('images_clear_val.npy')
images_noised_val = np.load('images_noised_val.npy')

In [4]:
#images_noised_train, images_noised_test, images_clear_train, images_clear_test = train_test_split(images_noised_tt,
#                                                                                                 images_clear_tt,
#                                                                                                 test_size=0.2,
#                                                                                                 random_state=42)

In [9]:
def dncnn_res(img_shape):

    input = Input(img_shape)
    hidden = 16
    x = Conv2D(128, kernel_size=(3,3), padding='same')(input)
    x = Activation('gelu')(x)
    for i in range(hidden):
        xx = Conv2D(128, kernel_size=(3,3), padding='same')(x)
        xx = BatchNormalization()(xx)
        xx = Activation('gelu')(xx)
        xx = Add()([x, xx])
        x = xx
    x = Conv2D(3, kernel_size=(3,3), padding='same')(x)
    out = Subtract()([input, x])

    model = Model(input, out)
    model.summary()
    return model

In [ ]:
model = dncnn_res((256,256,3))
reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss',
                                        factor=0.1,
                                        patience=5)

model.compile(optimizer=Adamax(learning_rate=1e-2), loss=ssim_l2)

history = model.fit(images_noised_tt[:5000], images_clear_tt[:5000],
                    epochs=30,
                    callbacks=[reduce_lr],
                    validation_split=0.2,
                    #validation_data=(images_noised_test, images_clear_test),
                    batch_size=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 256, 256, 12  3584        ['input_2[0][0]']                
                                8)                                                                
                                                                                                  
 activation_2 (Activation)      (None, 256, 256, 12  0           ['conv2d_2[0][0]']               
                                8)                                                            

                                                                                                  
 conv2d_9 (Conv2D)              (None, 256, 256, 12  147584      ['add_5[0][0]']                  
                                8)                                                                
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 256, 256, 12  512        ['conv2d_9[0][0]']               
 rmalization)                   8)                                                                
                                                                                                  
 activation_9 (Activation)      (None, 256, 256, 12  0           ['batch_normalization_7[0][0]']  
                                8)                                                                
                                                                                                  
 add_6 (Ad

                                8)                                'activation_15[0][0]']          
                                                                                                  
 conv2d_16 (Conv2D)             (None, 256, 256, 12  147584      ['add_12[0][0]']                 
                                8)                                                                
                                                                                                  
 batch_normalization_14 (BatchN  (None, 256, 256, 12  512        ['conv2d_16[0][0]']              
 ormalization)                  8)                                                                
                                                                                                  
 activation_16 (Activation)     (None, 256, 256, 12  0           ['batch_normalization_14[0][0]'] 
                                8)                                                                
          

In [ ]:
plot_history(history)

In [ ]:
y = predict_all(model, images_noised_val)